# Data Preprocessing
Prerequisites:
- Python experience, including a basing understanding of python syntax, loops, conditional statements, functions, and data types in python
- Some background in statistics is helpful

Goals for this session:
- Learn to perform basic exploratory data analysis (EDA) and data visualization
- Identify outliers, handle missing values, and perform other common data operations such as normalization, interpolation, and filtering
- Understand the intuition behind various preprocessing techniques for both categorical and continuous features
- Apply EDA and data preprocessing techniques to a novel data set without context


# Date fruit and Pumpkin seed dataset

# Techniques
In this code-along exercise, we will cover the following data processing techniques:
- Basic data views
- Renaming columns with `df.rename()`
- Filtering, viewing subsets with `df.loc[]`
- Sorting with `df.sort()`
- Merges with `pd.join` or `df.merge`
- Removing outliers with custom functions
- Normalization

[Pandas documentation](https://pandas.pydata.org/docs/reference/frame.html)

# Imports
By convention, we use the `as` in the import statement to alias `numpy` to `np`. Similarly, we alias `pandas` to `pd`. Another convention we will use is calling Pandas DataFrame objects `df`.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [ ]:
df.head()

In [ ]:
df.tail(10)

In [ ]:
df

Note that the Jupyter Notebook also prints out the number of rows and columns of the DataFrame when we just let it autoprint the DataFrame. We can calso view the shape of the DataFrame using `df.shape`. By convention, the first number in the tuple is the number of rows and the second number is the number of columns.

In [ ]:
df.shape

We can access the index and column axes of the DataFrame with `df.index` and `df.columns` respectively. The values of the DataFrame can be access with `df.values` which returns a numpy array.

In [ ]:
df.index

In [ ]:
df.columns

In [ ]:
df.values

Other conventions
- Specify one column with `"column name"`, specify multiple columns with `["list", "of", "column", "names"]`
- Index (`axis=0`) contains a unique identifier for each of the rows; Columns (`axis=1`) contain a unique identifier for each of the columns
- Most operations default to applying to the Index axis. It's best practice to specify the axis directly for clairty.

# Renaming Columns
Analyzing the column names, we notice a few things about the dataset. 

Of benefit:
1. Both datasets use many of the same features
2. Both datasets use the underscores to delimit words in the column names

Of concern:
1. Both datasets use different capitalization 
2. The feature of "equivalent diameter" in the pumpkin seeds to be "equivalent diameter squared" in the date fruits
3. The major and minor axis features are missing the word "length" for the data fruits


In [ ]:
df.columns

In [ ]:
pumpkin_seed_features = [
    'area',
    'perimeter',
    'major_axis_length',
    'minor_axis_length',
    'convex_area',
    'equiv_diameter',
    'eccentricity',
    'solidity',
    'extent',
    'roundness',
    'aspect_ratio',
    'compactness',
    'class'
]


In this section, we will deal with these concerns by renaming the columns using `df.rename()`.

`df.rename()` has two forms:
- `df.rename(mapper=function, axis="columns")` where we specify a function that is applied to all the column names. The function should result in all unique outputs when applied to all column names.
- `df.rename(mapper=Dict, axis="columns")` where we would specify a dictionary with keys as the old column names and values as the new column names. The columns to rename do not have to exist in the DataFrame; any keys in the dict that are not present will have no effect on the DataFrame and no error will be raised.

To deal with the capitalization, we will apply a function that transforms the string into all lowercase. To deal with the different names, we will make a dictionary that maps the lower-case incorrect names to correct ones.

In [ ]:
# Make mapper dict

# Make mapper function

# Apply to the data

# Viewing subsets
Just selecting data from a dataframe will not apply the operation to the dataframe, unless we assign it to a variable.

To view just a single columns, we can index the dataframe with that column name. Note that a single column will be returned as a `Series` object, which is similar to a dataframe, but with just one column. Therefore, some operations cannot be applied as with dataframes.

In [ ]:
# Select the area column


To view multiple columns, pass in a list of column names. When multiple columns are selected, a `DataFrame` is returned.

In [ ]:
# View the area and perimieter columns


There are two ways to select rows. If you know the row index or indicies, you can use `df.iloc[i]` or `df.iloc[[i, j, k]]` to select those rows.

If you know a condition on the values, you should use `df.loc[index_mask, columns]`. `index_mask` Boolean mask is a `(n,)` sized array of `True` or `False` values, while `columns` is a list of the columns. If `columns` are not specified, it will be assumed that all columns are selected, while rows will be filtered according to `index_mask`.

The mask can be made from taking a Series and applying a Boolean logic operator to it. For example, 
`mask = df[col] == value`. Multiple masks can be combined with AND (`&`) and OR (`|`) elementwise operators, with mask expressions in brackets.

In [ ]:
# Select rows 1, 3, and 5


In [ ]:
# View the area and perimeter of BERHI date fruits whose area is greater than 500_000


We can sort a dataframe by the values in a specific column using `df.sort_values(by=column_name)`. By default, sorting is by ascending order, so specifying `ascending=False` will sort in descending order.

In [ ]:
# Sort large BERHI dates by perimeter in descending order


Now that the column names are normalized and we know how to select specific columns, we can select the same subset of columns that the pumpkin seeds dataset has.

We could drop the unwanted columns one-by-one using `df.drop(labels=["column", "names"], axis=1)`. Some operations can also be applied to the same object by specifing the keyword `inplace=True`. By default, `inplace=False`, so changes will not be applied. Here, we will drop the unwanted columns `["meanrr", "meanrg", "meanrb"]` in place.

However, since we know the columns we want to keep, we could just select them and assign the result to `df` to keep them.

# Outlier removal

Outliers are data points that differ significantly from the mean distribution of the data. Often, outliers can represent incorrect measurements, although domain knowledge is often required to interpret the meaning of outliers. We will only analyze the feature columns (e.g. not `class`) for outliers, since they're all continuous values, whereas `class` is categorical.

In this example, we can view the distributions of each of the features and note that there is an outlier with the aspect ratio with an abnormally large value, where we would typically expect this feature to be between 1-3. There are some compactness and roundness values near 0, where we expect these to be near 1. The solidity feature also has a long left tail, so some lower values might also be considered outliers.

There are many ways to remove outliers, like using heuristic rules. There are also statistical methods. In this example, we'll compute a robust Z-score using $z' = \frac{x - \tilde{x}}{1.4826 \times \text{MAD}(x)}$ where $\tilde{x}$ is the median of $x$, MAD is the median absolute deviation, and 1.4826 is a scale factor so that the MAD approximates the standard deviation of a normal distribution. 

We will reject outliers as having a robust Z-score of $|z'| > 6$.

In [ ]:
# Plotting the distribution
fig, axs = plt.subplots(3, 4, figsize=(8, 6))

feature_cols = df.columns.drop("class")

for col, ax in zip(feature_cols, axs.flatten()):
    ax.hist(df[col].values, bins=20)
    ax.set_xlabel(col)

plt.tight_layout()
plt.show()

In [ ]:
# Compute some stats
from scipy.stats import median_abs_deviation

df_score = df.copy(deep=True)


In [ ]:
# Apply robust Z-score to each column
for col in feature_cols:
    

In [ ]:
df_score

In [ ]:
# mask is a (n, d) array of (0, 1) values
mask = (df_score[feature_cols] > 6) | (df_score[feature_cols] < -6)
mask.sum(axis="rows")

In [ ]:
# Summing mask along columns reveals which rows are outliers
df.loc[mask.sum(axis="columns") != 0].sort_values(by="area")

In [ ]:
# Remove outliers by only selecting non-outlier indices


In [ ]:
# Plotting the distribution after outlier removal
fig, axs = plt.subplots(3, 4, figsize=(8, 6))

for col, ax in zip(feature_cols, axs.flatten()):
    ax.hist(df[col].values, bins=20)
    ax.set_xlabel(col)

plt.tight_layout()
plt.show()

# Feature normalization

As discussed earlier, machine learning models require their inputs to be in vectors. Additionally, a lot of ML models assume that the input data is roughly normally distributed with 0 mean and unit standard deviation, or that the data is scaled between [-1, 1]. By preparing data to have a standard normal distribution generally improves model performance compared to having un-normalized features.

Scikit-learn has a handy interface to apply common normalization techniques like standard scaling and min-max scaling to dataframes.

We only apply this normalization to the feature columns, e.g. not the `class` column. Later, we will talk about how to encode categorical variables like `class` for ML models.

In [ ]:
# Select feautre columns only

In [ ]:
# Apply standard scaling

We can see that the mean and standard deviation of the features are now 0 and 1 respectively.

In [ ]:
X_df_trans.agg(["mean", "std"]).round(3)

Finally, we can retreive the feature matrix and pass this to a machine learning model as the input features! Note, we would still need to encode the labels (`df["class"]`) as a vector first before being able to do supervised learning.

In [ ]:
# Retreive feature matrix as df values

# Working with Different Data Types

The candy dataset contains survey results. Respondents were asked to rate various Halloween candies as 'Meh', 'Joy', or 'Despair' based on how happy they would be to receive a particular candy.

In [ ]:
candy = pd.read_excel('../data/candy.xlsx')

In [ ]:
candy.head()

## Ordinal Categoricals

The candy dataset contains mostly categorical data. Many machine learning models require vector inputs with numerical data. So, a very common operation is to encode categorical variables, effectively converting them into integer format.

Ordinal categoricals are discrete categories that have some order. Think "small", "medium", "large". One technique for converting nominal categorical variables to numeric values is label encoding.

One particular type of candy can have 3 ratings. These ratings have an implicit order.

A natural solution would be to assign DESPAIR a value of -1, MEH a value of 0, and JOY a value of 1.

In [ ]:
from sklearn.preprocessing import LabelEncoder

## Nominal Categoricals

Nominal categoricals have no order, so concepts such as mean, min, and max have no interpretation. One particularly popular method of encoding, known as One-Hot Encoding involves representing categorical variables as binary columns.

## Converting to Categorical

Suppose you are interested in comparing candy ratings across age groups. Looking at ratings for each individual age does not make practical sense, so you could bin the age column into categories.

In [ ]:
candy['Q3: AGE'].unique()

In [ ]:
age_groups = candy[candy['Q3: AGE'].apply(lambda x: str(x).isnumeric())]['Q3: AGE']

In [ ]:
age_groups.apply(lambda x: round(x, -1))

# Handling Missing Values

Not all missing values are equal. Consider arbitrary survey data taken from the general population.

**Missing Not At Random** - when a value is missing for a reason related to the true value. (Ex: if a survey responding chooses not to disclose their income, this could be because they have an abnormally high or low income)

**Missing at Random** - when a value is missing for a reason related to another observed variable. (Ex: many age values are missing for survey respondents of a particular gender)

**Missing Completely at Random** - when there's no patterns in the missing values.

In [ ]:
solar_df = pd.read_csv("../data/solarenergy.csv")

In [ ]:
solar_df.head()

## Deletion

- Column deletion: removing a column that has too many missing values and is non-essential for your model
- Row deletion: removing rows with missing values, ideally if the missing values are Missing At Random, to avoid biasing your model

In [ ]:
solar_df.dropna()

In [ ]:
solar_df.head()

Two ways of making row deletion persistent in a DataFrame.

In [ ]:
# solar_df.dropna(inplace=True)
# solar_df = solar_df.dropna(subset=['solar radiation'])

In [ ]:
solar_df.drop('solar radiation')
#solar_ df.drop('solar radiation', axis=1) # set axis=1 to drop a row!

## Imputation

- Fill missing values with their defaults (empty string, zero, etc...)
- Fill missing values with the mean, median, or mode
- Backward or forward fill
- Imputation risks injecting your own bias and adding noise to the data, and should be performed with caution

In [ ]:
# don't worry too much about this line
null_indices = solar_df.loc[pd.isna(solar_df['solar radiation']), :].index
interp = interp_df.iloc[null_indices]

plt.plot(solar_df['time'].values[:100], solar_df['solar radiation'].values[:100])
plt.scatter(interp.time.values[:10], interp['solar radiation'].values[:10], color='orange')